# Tracking HAB

In [1]:
# standard imports
import os

## Reading data from the file

In [2]:
data = open(os.path.join(os.path.realpath('../data'), 'Log-Data.txt' ), mode='r', encoding='utf-8').read()

## Cleaning the data

In [3]:
data = data.split('\n')

In [4]:
time = []
longitude = []
latitude = []
voltage = []
altitude = []
iteration = []

for i in range(len(data)):
    if i%2==0:
        time.append(data[i].split('$')[0])
    else:
        clean_array = data[i].strip().split('!')
        clean_text = "".join(clean_array);
        recleaned_text = clean_text.split(" ")
        recleaned_text.pop() # removing the last element of the array
        location = recleaned_text[0]
        # clean location
        cleaned_location = location.split('/')
        long = cleaned_location[0][:7]
        long = float(long[:2]) + float(long[2:])/60
        if "S" in cleaned_location:
            long = -long;
        longitude.append(long)
        
        lat = cleaned_location[1][:8]
        lat = float(lat[:3]) + float(lat[3:])/60
        if "W" in cleaned_location:
            lat = -lat
        latitude.append(lat)
        
        volt = recleaned_text[2] + " V"
        voltage.append(volt)
        if recleaned_text[4]:
            alt = recleaned_text[4] + " ft"
        else: 
            alt = "0 ft"
        altitude.append(alt)
        iteration.append(recleaned_text[5])

## Reformatting in table form

In [5]:
# importing pandas
import pandas as pd

# creating table from the data

data = {'Time': time,
        'Longitude': longitude,
        'Latitude': latitude,
        'Voltage': voltage,
        'Altitude': altitude,
        'Iteration': iteration
       }
df = pd.DataFrame(data)
df

,Time,Longitude,Latitude,Voltage,Altitude,Iteration
0,16:14:01,0.000000,0.000000,4.16 V,0 ft,2
1,16:15:00,0.000000,0.000000,4.16 V,0 ft,1
2,16:20:19,0.000000,0.000000,4.16 V,0 ft,1
3,16:21:40,0.000000,0.000000,4.16 V,0 ft,1
4,16:23:12,0.000000,0.000000,4.16 V,0 ft,1
5,16:25:09,0.000000,0.000000,4.16 V,0 ft,1
6,16:26:09,0.000000,0.000000,4.16 V,0 ft,1
7,16:28:10,0.000000,0.000000,4.16 V,0 ft,2
8,16:34:11,0.000000,0.000000,4.16 V,0 ft,5
9,16:40:12,27.622667,85.537667,4.24 V,1514 ft,8


## Mapping the co-ordinates

In [6]:
import folium

In [7]:
display_map = folium.Map(width=800, height=600, zoom_start=13)
last_location = longitude[-1], latitude[-1]
for i in range(len(longitude)-1):
    if longitude[i] or latitude[i]:
        location = longitude[i], latitude[i]
        if location != last_location:
            folium.Marker(
                location, popup=f"Time: {time[i]}\nBattery: {voltage[i]}\nAltitude: {altitude[i]}\nIteration: {iteration[i]}", icon = folium.Icon(icon="cloud")
            ).add_to(display_map)
        else:
            folium.Marker(
                location, popup=f"Status: Landed, Time: {time[i]}\nBattery: {voltage[i]}\nAltitude: {altitude[i]}\nIteration: {iteration[i]}", icon = folium.Icon(color="red", icon="info-sign")
            ).add_to(display_map)
            folium.Circle(last_location, radius = 100, color="red", fill=True, fill_color="#ffcccb" ).add_to(display_map)
        

        
display_map